Primeiro passo importando as bibliotecas

In [ ]:
import pandas as pd
import csv #importando csv
import requests
from bs4 import BeautifulSoup

- A pesquisa ficou adestrita a região do vale do Jauru

- Região vale do Jauru

        Araputanga
        Figueirópolis d'Oeste
        Glória d'Oeste
        Indiavaí
        Jauru
        Lambari d'Oeste
        Mirassol d'Oeste
        Porto Esperidião
        Reserva do Cabaçal
        Rio Branco
        Salto do Céu
        São José dos Quatro Marcos


In [ ]:

cidade_link = open('src/arquivos.csv')

implementa = csv.DictReader(cidade_link)
#a leitura só funcionara com os parametros abaixo com o parametro acimas

for inplemente in implementa:
    #lendo arquivo python para rastrear o separador
    carregar_separador = open(inplemente['link'],encoding='ISO-8859-1')
    separa = csv.Sniffer().sniff(carregar_separador.read())
    separador = separa.delimiter
    print("Tipo de separador definido foi --> ", separador)
    #---------------------------------------
    #dividindo municipio para colocar na variavel de nome    
    dividi_municipio = inplemente['municipio'].split(' ')# o codigo dividi o nome do municipio
    #----------------------------------------------
    
    # Carregando csv no pandas 
    servidores = pd.read_csv(inplemente['link']#chama o arquivo csv 
                             ,sep=separador #informa o separador do arquivo
                             ,encoding='ISO-8859-1', #informa a codificação do arquivo
                             error_bad_lines=False,
                             engine='python') #trata os erros nas linhas
    nome_variavel = 'servidores_{}'.format(dividi_municipio[0])     #alterando o nome da variavel
    globals()[nome_variavel] = servidores
    globals()[nome_variavel]['Municipio'] = inplemente['municipio']
    print(nome_variavel)
    print('-------------------------')



- foi necessario criar uma variavel com todos os servidores dos municipios da região oeste

In [ ]:
servidores_municipio_região_oeste = servidores_Lambari, servidores_Indiavaí, servidores_Jauru, servidores_Mirassol, servidores_Porto, servidores_Reserva, servidores_São, servidores_Figueirópolis,servidores_Salto, servidores_Rio, servidores_Gloria, servidores_Araputanga

In [ ]:
resultado = pd.concat(servidores_municipio_região_oeste)
#salvando os dados dos servidores da região oeste em CSV
resultado.to_csv('servidores_selecionado.csv')

In [ ]:
#exibindo os dados referente a região oeste 
resultado.head()

In [ ]:
# Objetivo ->    Separando os dados desejados
# Motivo   ->    A tabela estava muito populada com dados que não eram necessarios
# Ação     ->    Ver quais tabelas são necessarias e isolar somente a elas    
resultado = resultado[['Nome', 'Cargo', 'Cargo inicial', 'Municipio']]
resultado.head()

# Objetivo
- Baixar dados do auxilio de forma descentralizada

#### FONTE DE DADOS
- SITE BRASIL IO 
* https://brasil.io/dataset/govbr/auxilio_emergencial/

#### METODO
- baixar os dados por municipios e salvar
- **DIFICULDADES**
    - O ARQUIVO VINHA COM MASCARA ENTÃO ORDENOU POR NUMERO PARA IMPLEMENTAR UMA BUSCA COM O FOR 

Implementando codigo para unir dados baixados do site brasil io

In [ ]:
for i in range(1,12):
    print('df'+str(i))
    df = pd.read_csv("src/dados_brasil_io/govbr-{}.csv".format(i))
    nome_variavel = 'df{}'.format(i)
    globals()[nome_variavel] = df

**Unindo os dataframes**

In [ ]:
auxilio_emergencial = df1, df2, df3, df4, df5, df6, df7, df8, df9, df10 ,df11
auxilio_emergencial_vale_jauru = pd.concat(auxilio_emergencial)

In [ ]:
auxilio_emergencial_vale_jauru

In [ ]:
auxilio_emergencial_vale_jauru = auxilio_emergencial_vale_jauru.rename(columns={'beneficiario':'Nome'})
auxilio_emergencial_vale_jauru.head()

In [ ]:
# Objetivo ->    Separando os dados desejados
# Motivo   ->    A tabela estava muito populada com dados que não eram necessarios
# Ação     ->    Ver quais tabelas são necessarias e isolar somente a elas    
resultado = resultado[['Nome', 'Cargo', 'Cargo inicial', 'Municipio']]
auxilio_emergencial_vale_jauru_trat =auxilio_emergencial_vale_jauru[['municipio', 'cpf_beneficiario', 'enquadramento', 'Nome']]

In [ ]:
ser_rec_aux = auxilio_emergencial_vale_jauru_trat.merge(resultado, on='Nome')

In [ ]:
ser_rec_aux.head()

# Observações
- Conforme observa-se na busca há servidores que tem mesmo nome (homonimos) em diversas Unidade Gestoras dessa forma faz-se necessario fazer uma comparação no mesmo municipio.
- tem como objetivo isolar servidores pelo municipio que mesmo recebeu o auxilio e assim poder realmente comparar.
- Há formula abaixo não evita homonimos mas minimiza 

In [ ]:
# Abreviatura
# ser_rec_aux_mun  = Servidores que receberan Auxilio Emergencial por municipio
ser_rec_aux_mun = ser_rec_aux[ser_rec_aux['municipio'] == ser_rec_aux['Municipio']]
ser_rec_aux_mun.head()


In [ ]:
ser_rec_aux_mun.info()

In [ ]:
ser_rec_aux_mun.head()

# _________________________________________
## Objetivo pegar os dados dos municipios com base nos nomes registrados no TCE-MT e do IPEA 
- NÃO FOI ENCONTRADO NENHUMA BASE DE DADOS ONDE E POSSIVEL CORRELACIONAR O CODIGO DO IBGE COM OS DADOS DOS CODIGOS DO TCE.
- TODAVIA É POSSIVEL ISOLAR A AREA PELO NOME DO TCE E CRIAR UMA TABELA DE CORRELAÇÃO PELO NOME
    - Metodo - implementar correlação somente para região oeste, assimm buscou dados de correlação de região sendo possivel achalas no site do IPEA no site:https://www.ipea.gov.br/ipeageo/bases.html
        - baixei os dados referente ao estado de Mato Grosso - MT em formato excel
    - Para pegar os nomes cadastrado no site do Tribunal de Contas do Estado de Mato Grosso, a busca deu-se no link https://servicos.tce.mt.gov.br/tabela-interna
        - Baixei as tabelas:
            - ***MUNICIPIO_MT*** - Cadastro dos municipios de Mato Grosso
            - ***TIPO_ENTIDADE*** - cadastro de tipo de unidade Gestora
            - ***UG*** - Unidades Gestoras Cadastradas
            

In [ ]:
# Carregando e exibindo dados dos Municipios de Mato Grosso (MT)
df_municiopios_mt = pd.read_csv('src/dados_tce_mt/APLIC/MUNICIPIO_MT.csv', sep=';')
df_municiopios_mt.head()

In [ ]:
# Carregando dados de Mato grosso 
# -----------Comentarios necessarios---------------
# - Tabelas Excel normamente vem com abas nomeada assim foi necessario verificar a que melhor atendia aos
#   quesistos a ser levantado esse nome é lançado no campo sheet_name
#---------------------------------------------------
df_dados_ipea = pd.read_excel('src/dados_ipea/MT_Mun97_region.xls', sheet_name='PIB  municipal (2010)')
df_dados_ipea 

In [ ]:
#tratando e selecionando somente os dadnos necessarios da tabela do IPEA
df_dados_ipea = df_dados_ipea[['Código IBGE do Município', 'Código IBGE da Microregião','Nome da Microregião']]
df_dados_ipea

In [ ]:
#selecionando dados do vale do Jauru
df_dados_ipea = df_dados_ipea[df_dados_ipea['Nome da Microregião']== 'Jauru']
df_dados_ipea

In [ ]:
df_dados_ipea = df_dados_ipea.rename(columns={'Código IBGE do Município':'MUN_CODIGO'})
df_dados_ipea

In [ ]:
df_municiopios_mt.info()

In [ ]:
df_dados_ipea.info()

In [ ]:
#df_municiopios_mt['MUN_CODIGO'] = df_municiopios_mt['MUN_CODIGO'].fillna(0.0).astype(int)
#df_municiopios_mt.info()

In [ ]:
df_municiopios_mt.head()

In [ ]:
df_municiopios_mt[df_municiopios_mt['MUN_NOME'] == 'JAURU']

In [ ]:
df_municipios_com_id = df_municiopios_mt.merge(df_dados_ipea, on='MUN_CODIGO')

In [ ]:
df_municipios_com_id = df_municipios_com_id[['MUN_CODIGO', 'Nome da Microregião','MUN_NOME', 'GENTILICO']]
df_municipios_com_id.head()

In [ ]:
df_ug_tce = pd.read_csv('src/dados_tce_mt/APLIC/UG.csv', sep=';')
df_ug_tce.head(60)

In [ ]:
dados_jose = pd.read_excel('src/dados_tce_mt/Leandro DSA.xlsx')
dados_jose

In [ ]:
selecionando_municipios = df_municipios_com_id.merge(dados_jose, on='MUN_CODIGO')
selecionando_municipios

In [ ]:
#isolando somente as prefeituras
prefeitura_nome = []
for x in selecionando_municipios['ENT_NOME']:
    prefeitura = x.count('PREFEITURA')
    if prefeitura != 0:
        #print(x)
        prefeitura_nome.append(x)
            
prefeitura_nome
prefeituras = pd.DataFrame(prefeitura_nome,columns=['ENT_NOME'])

In [ ]:
seleciona_prefeitura = selecionando_municipios.merge(prefeituras, on='ENT_NOME')
seleciona_prefeitura

# Implementando busca automatizada no site do Tribunal de Contas do Estado de Mato Grosso na parte de pessoal 
- site: https://cidadao.tce.mt.gov.br/pessoal
- Pois lá tem parte do CPF que pode ser consultada
 <img src='src/img/Screenshot_2020-07-23 TCEMT Espaço do Cidadão - Pessoal.png'>

In [ ]:
# Durante pesquisa foi possivel verificar que o Tribunal de Contas do Estado de 
# Mato Grosso tem dados de 2011 a 2020
# Solução implementar um range de 2011 a 2021
# Lembrando que python sempre le um numero  a menos que informado pois a contagem inicia em 0
ato_pessoal_tce_anos = list(range(2011,2021))
print(ato_pessoal_tce_anos)


In [ ]:
#definindo url
url = 'https://cidadao.tce.mt.gov.br/pessoal/lista?exercicio=2011&situacao=situacao[]%3DEFETIVO%26situacao[]%3DELETIVO%26situacao[]%3DLIVRE%2BNOMEA%25C7%25C3O%2BE%2BEXONERA%25C7%25C3O&municipio=510125&ent_codigo=1121375&lotacao='

In [ ]:
for x in ato_pessoal_tce_anos:
    r = requests.get('https://cidadao.tce.mt.gov.br/pessoal/lista?exercicio={}&situacao=situacao[]%3DEFETIVO%26situacao[]%3DELETIVO%26situacao[]%3DLIVRE%2BNOMEA%25C7%25C3O%2BE%2BEXONERA%25C7%25C3O&municipio=510125&ent_codigo=1121375&lotacao='.format(x))
    print(x)
    


In [ ]:
dataframetce = pd.read_html(r.text)

In [ ]:
dataframetce[1]

In [ ]:
soup = BeautifulSoup(r.content, 'html.parser')
print(soup.prettify())

In [ ]:
soup.select('thead tr')